In [1]:
# =========================
# IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_plots import *

# =========================

c:\Users\sal_f\anaconda3\envs\env_ggs0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

In [1]:
import xarray as xr

data = xr.open_dataset('C:/Users/sal_f/OneDrive/Desktop/STF-0/!-GGS/GGS_Scripts/data/reprocess/Sentinel Leg 5_CMEMS_DepthAverage_20240329T00Z.nc')
data

<xarray.Dataset>
Dimensions:        (time: 1, lat: 361, lon: 2617)
Coordinates:
  * lat            (lat) float32 -60.0 -59.92 -59.83 ... -30.17 -30.08 -30.0
  * lon            (lon) float32 -53.0 -52.92 -52.83 ... 164.8 164.9 165.0
Dimensions without coordinates: time
Data variables:
    u_depth_avg    (time, lat, lon) float64 ...
    v_depth_avg    (time, lat, lon) float64 ...
    mag_depth_avg  (time, lat, lon) float64 ...
    dir_depth_avg  (time, lat, lon) float64 ...
Attributes:
    model_datetime:  2024-03-29T00:00:00
    model_name:      CMEMS

---

In [2]:
# =========================
config_name = "ugos"

config = GGS_config_import(config_name)

target_datetime = config['MISSION'].get('target_date')
if not target_datetime:
    print("Issue with target datetime. Using current datetime.")
    target_datetime = dt.datetime.now(dt.timezone.utc)

root_directory = GGS_config_process(config, path="default")

datetime_list = [target_datetime.replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')]
datetime_index = datetime_list[0]

glider_data_flag = None
if config['PLOT'].get('show_gliders'):
    min_lat, min_lon = config['MISSION']['extent'][0]
    max_lat, max_lon = config['MISSION']['extent'][1]
    search_extent = [min_lon, max_lon, min_lat, max_lat]
    glider_data_flag = acquire_gliders(
        extent=search_extent,
        target_date=target_datetime,
        date_delta=dt.timedelta(days=1),
        requested_variables=["time", "longitude", "latitude", "profile_id", "depth"],
        print_vars=False,
        target="all",
        request_timeout=5,
        enable_parallel=False
    )

sub_directory_plots = os.path.join(root_directory, "plots", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_plots, exist_ok=True)
sub_directory_data = os.path.join(root_directory, "data", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_data, exist_ok=True)

# =========================

config_flag = config

enable_rtofs_flag = config_flag['MODEL'].get('enable_rtofs')
enable_cmems_flag = config_flag['MODEL'].get('enable_cmems')
enable_gofs_flag = config_flag['MODEL'].get('enable_gofs')
subset_extent_flag = config_flag['MODEL'].get('subset_extent')
subset_depth_flag = config_flag['MODEL'].get('subset_depth')
chunk_flag = config_flag['MODEL'].get('chunk')

latitude_qc_flag = config_flag['PLOT'].get('latitude_qc')
longitude_qc_flag = config_flag['PLOT'].get('longitude_qc')
density_flag = config_flag['PLOT'].get('density')
mag1_flag = config_flag['PLOT'].get('mag1')
mag2_flag = config_flag['PLOT'].get('mag2')
mag3_flag = config_flag['PLOT'].get('mag3')
mag4_flag = config_flag['PLOT'].get('mag4')
mag5_flag = config_flag['PLOT'].get('mag5')
tolerance_flag = config_flag['PLOT'].get('tolerance')
show_route_flag = config_flag['PLOT'].get('show_route')
show_eez_flag = config_flag['PLOT'].get('show_eez')
show_qc_flag = config_flag['PLOT'].get('show_qc')
manual_extent_flag = config_flag['PLOT'].get('manual_extent')

# =========================

model_datasets = []
    
if enable_rtofs_flag:
    try:
        rtofs = RTOFS()
        rtofs.rtofs_load(config_flag, datetime_index, subset_extent=subset_extent_flag, subset_depth=subset_depth_flag)
        rtofs.rtofs_save(config_flag, sub_directory_data, save_data=False, save_qc=False)
        rtofs_model_data = rtofs.data
        
        rtofs_depth_average, rtofs_bin_average = interpolate_rtofs(config_flag, sub_directory_data, rtofs_model_data, chunk=chunk_flag, save_depth_average=True, save_bin_average=False)

        rtofs_datasets = (rtofs_model_data, rtofs_depth_average, rtofs_bin_average)
        model_datasets.append(rtofs_datasets)
    except Exception as e:
        print(f"Error during RTOFS processing: {e}")

if enable_cmems_flag:
    try:
        cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
        cmems.cmems_load(config_flag, datetime_index, subset_extent=subset_extent_flag, subset_depth=subset_depth_flag)
        cmems.cmems_save(config_flag, sub_directory_data, save_data=False, save_qc=False)
        cmems_model_data = cmems.data
        
        cmems_depth_average, cmems_bin_average = interpolate_cmems(config_flag, sub_directory_data, cmems_model_data, chunk=chunk_flag, save_depth_average=True, save_bin_average=False)
        
        cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
        model_datasets.append(cmems_datasets)
    except Exception as e:
        print(f"Error during CMEMS processing: {e}")

if enable_gofs_flag:
    try:
        gofs = GOFS()
        gofs.gofs_load(config_flag, datetime_index, subset_extent=subset_extent_flag, subset_depth=subset_depth_flag)
        gofs.gofs_save(config_flag, sub_directory_data, save_data=False, save_qc=False)
        gofs_model_data = gofs.data

        gofs_depth_average, gofs_bin_average = interpolate_gofs(config_flag, sub_directory_data, gofs_model_data, chunk=chunk_flag, save_depth_average=True, save_bin_average=False)

        gofs_datasets = (gofs_model_data, gofs_depth_average, gofs_bin_average)
        model_datasets.append(gofs_datasets)
    except Exception as e:
        print(f"Error during GOFS processing: {e}")

# =========================



### IMPORTING GGS CONFIGURATION: ugos ###

Locating glider: None
Success!

### PROCESSING GGS CONFIGURATION ###

Configuration:

--> MISSION
mission_name: UGOS
target_date: 2024-03-26T03:38:10+0000
max_depth: 1000
extent: [(15, -90), (30, -78)]
GPS_coords: None
glider_id: None

--> MODEL
single_datetime: True
enable_rtofs: True
enable_cmems: True
enable_gofs: True
subset_extent: True
subset_depth: True
chunk: True

--> PLOT
latitude_qc: 21.175
longitude_qc: -86.185
density: 2
mag1: 0.0
mag2: 0.2
mag3: 0.3
mag4: 0.4
mag5: 0.5
tolerance: 15.0
show_gliders: True
show_route: False
show_eez: True
show_qc: False
manual_extent: None

--> DATA
bathymetry_path: c:\Users\sal_f\OneDrive\Desktop\STF-0\!-GGS\GGS_Scripts\data\bathymetry\GEBCO_2023_sub_ice_topo.nc
eez_path: c:\Users\sal_f\OneDrive\Desktop\STF-0\!-GGS\GGS_Scripts\data\eez\eez_boundaries_v12.shp


### ACQUIRING GLIDER DATASETS ###

Start time (UTC): 2024-03-26 03:38:10.608276
Found 4 Glider Datasets within search window.
Glider Indexe

In [ ]:
GGS_plot_magnitudes(config_flag,
                    sub_directory_plots,
                    datetime_index,
                    model_datasets,
                    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
                    density=density_flag,
                    gliders=glider_data_flag,
                    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
                    manual_extent=manual_extent_flag)
GGS_plot_threshold(config_flag,
                    sub_directory_plots,
                    datetime_index,
                    model_datasets,
                    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
                    density=density_flag,
                    mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
                    gliders=glider_data_flag,
                    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
                    manual_extent=manual_extent_flag)
GGS_plot_advantage(config_flag,
                    sub_directory_plots,
                    datetime_index,
                    model_datasets,
                    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
                    density=density_flag,
                    tolerance=tolerance_flag,
                    mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
                    gliders=glider_data_flag,
                    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
                    manual_extent=manual_extent_flag)
GGS_plot_profiles(config_flag,
                    sub_directory_plots,
                    datetime_index,
                    model_datasets,
                    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
                    threshold=0.5)
GGS_plot_profiles_old(config_flag,
                    sub_directory_plots,
                    datetime_index,
                    model_datasets,
                    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
                    threshold=0.5)

---